<div style="text-align: right"><i>Ivy Zhang<br>2024</i></div>

## Vose Alias Method

I first saw this mentioned in the [Node2Vec paper](https://arxiv.org/abs/1607.00653) where they used this technique to sample from a discrete distribution in $O(1)$!

For an extremely well written essay on this method: you should go read [Darts, Dice, & Coin](https://www.keithschwarz.com/darts-dice-coins/). I've tried to provide the cleanest solution I can think of in Python here :)

In [105]:
import matplotlib.pyplot as plt
import random
import math
from typing import *

In [106]:
def precompute_tables(sample: List[float]) -> Tuple[List[float], List[int]]:
    """
    Precomputes tables to be able to sample from in O(1) later.
    
    Takes in an arbitrary list of floats and converts them to probability. It
    then returns a Probs array and Alias array.
    """
    n = len(sample)

    res_alias = [None for _ in range(n)]
    res_probs = [0 for _ in range(n)]

    # These are stacks like mentioned in array
    small_idxs = []
    large_idxs = []

    # First convert inputs into probabilities
    sample_sum = sum(sample)
    probs = [x / sample_sum for x in sample]

    # We then scale it so that average prob (1/n) would become 1
    probs = [x * n for x in probs]

    # Create your stacks
    for i, p in enumerate(probs):
        if p < 1:
            small_idxs.append(i)
        else:
            large_idxs.append(i)

    # Continue pairing large and small pairings to fill columns
    while len(small_idxs) and len(large_idxs):
        si = small_idxs.pop()
        li = large_idxs.pop()

        res_probs[si] = probs[si]
        res_alias[si] = li
        probs[li] = (probs[li] + probs[si] - 1.0)

        if probs[li] < 1.0:
            small_idxs.append(li)
        else:
            large_idxs.append(li)

    # These are just to deal with numerical instability
    # Anything left here should just be it's own column
    while len(small_idxs):
        si = small_idxs.pop()
        res_probs[si] = 1.0

    while len(large_idxs):
        si = large_idxs.pop()
        res_probs[si] = 1.0

    return (res_probs, res_alias)

In [107]:
def generate(probs, alias):
    """
    Generates a random element with a fair dice and a biased coin toss.
    """
    n = len(probs)
    idx = math.floor(random.random() * n)
    flip = random.random()
    return idx if flip < probs[idx] else alias[idx]
    

In [108]:
num_gens = 10_000_000
inputs = [20, 10, 30, 5]
p_table, alias_table = precompute_tables(inputs)

res = [0 for _ in range(len(inputs))]

for _ in range(num_gens):
    res[generate(p_table, alias_table)] += 1

res_scaled = [x/sum(res) for x in res]
inputs_scaled = [x/sum(inputs) for x in inputs]

In [109]:
print('Initial distribution:', inputs_scaled)
print(f'Dist after {num_gens} samples:', res_scaled)

Initial distribution: [0.3076923076923077, 0.15384615384615385, 0.46153846153846156, 0.07692307692307693]
Dist after 10000000 samples: [0.3074491, 0.1538353, 0.4617926, 0.076923]
